In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# RAG Engine in Vertex AI


| | |
|-|-|
| Author(s) | [Holt Skinner](https://github.com/holtskinner) |

### Install Vertex AI SDK and other required packages


In [1]:
%pip install --upgrade --user --quiet google-cloud-aiplatform

### Authenticate to GCP

In [2]:
import sys
from IPython.display import Markdown, display

PROJECT_ID="ai-hangsik"
LOCATION="us-central1"
STAGING_BUCKET = "gs://reasoning_7424"
MODEL = "gemini-1.5-flash"

# For only colab user, no need this process for Colab Enterprise in Vertex AI.
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id=PROJECT_ID)

# set project.
!gcloud config set project {PROJECT_ID}

Updated property [core/project].


In [3]:
import vertexai
from IPython.display import Markdown
from vertexai.preview.generative_models import GenerativeModel, Tool

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Tool,
)

# Preview
from vertexai.preview import (
    rag,
    reasoning_engines)

from vertexai.preview.generative_models import (
    grounding)

# model initialization
vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)
model = GenerativeModel(MODEL)

### Create a RAG Corpus

In [4]:
# Currently supports Google first-party embedding models

#EMBEDDING_MODEL = "publishers/google/models/text-embedding-004"
EMBEDDING_MODEL="publishers/google/models/text-multilingual-embedding-002"

embedding_model_config = rag.EmbeddingModelConfig(publisher_model=EMBEDDING_MODEL)

rag_corpus = rag.create_corpus(
    display_name="my-rag-corpus", embedding_model_config=embedding_model_config
)

### Check the corpus just created

In [ ]:
#rag.list_corpora()

### Import files from Google Cloud Storage


In [5]:

INPUT_GCS_BUCKET_LIST = ["gs://daou_office_manual/manual_org/DaouOffice 클라우드 관리자 가이드.pdf",
        "gs://daou_office_manual/manual_org/DaouOffice 클라우드 서비스 가이드 3.5.21.pdf",
        ]

response = rag.import_files(
    corpus_name=rag_corpus.name,
    paths=INPUT_GCS_BUCKET_LIST,
    chunk_size=1024,  # Optional
    chunk_overlap=100,  # Optional
    max_embedding_requests_per_min=900,  # Optional
)

### Optional: Perform direct context retrieval

In [6]:
# Direct context retrieval
response = rag.retrieval_query(
    rag_resources=[
        rag.RagResource(
            rag_corpus=rag_corpus.name,
            # Optional: supply IDs from `rag.list_files()`.
            # rag_file_ids=["rag-file-1", "rag-file-2", ...],
        )
    ],
    text="다우 오피스 매뉴얼에서 대쉬보드에 대해서 설명해주세요.",
    similarity_top_k=10,  # Optional
    vector_distance_threshold=0.5,  # Optional
)
print(response)

# Optional: The retrieved context can be passed to any SDK or model generation API to generate final results.
# context = " ".join([context.text for context in response.contexts.contexts]).replace("\n", "")

contexts {
  contexts {
    source_uri: "gs://daou_office_manual/manual_org/DaouOffice 클라우드 서비스 가이드 3.5.21.pdf"
    source_display_name: "DaouOffice 클라우드 서비스 가이드 3.5.21.pdf"
    text: "○ 삭제 - 사용하지 않는 기기 정보를 리스트에서 삭제할 수 있습니다.개인정보 수정 | 49\r\n사용하고 있는 기기 정보가 보이지 않는 경우, 앱 로그아웃 후 다시 로그인해 주시면 기기정보가 등록됩니\r\n다.2. 홈\r\n51\r\n2.1 대시보드란\r\nDaouOffice에 접속하면 보이는 첫 화면이 대시보드입니다. 대시보드는 여러 개의 가젯으로 이루어져 있습니다. 가젯\r\n은 특정 메뉴로 바로가는 기능이나 동적으로 바뀌는 정보를 보여주기 때문에 대시보드에 가젯을 적절히 구성하면, 필\r\n요한 정보를 실시간으로 확인할 수 있어 편리합니다.\r\n대시보드에는 2가지 종류가 있습니다.\r\n● 전사 대시보드 - 가장 기본이 되는 대시보드로 전사 Portal로 표시됩니다. 각 메뉴에서도 언제나 화면 왼쪽 상단\r\n에 있는 로고를 클릭하면 전사 대시보드로 이동합니다.\r\n전사 대시보드는 기본적으로 모든 사용자에게 똑같이 보이며, 전사 대시보드 운영자가 관리할 수 있습니다. 전사\r\n대시보드 설정에 따라 사용자가 전사 대시보드에도 가젯을 추가하거나 삭제할 수도 있습니다.\r\n● 개인 대시보드 - 사용자가 직접 대시보드를 추가하여 필요한 가젯을 원하는 대로 배치하여 사용할 수 있는 대시\r\n보드입니다.\r\n대시보드 운영자 및 권한에 대한 문의는 사이트 관리자에게 하시기 바랍니다.\r\n대시보드에 추가할 수 있는 가젯 종류는 다음과 같습니다.\r\n● HTML 편집기 - HTML을 이용하여 사용자가 가젯을 직접 추가할 수 있습니다.\r\n● Quick Menu - 메일이나 게시물 작성, 일

### [Preview] Create RAG Retrieval Tool

In [7]:
# Create a tool for the RAG Corpus
rag_retrieval_tool = Tool.from_retrieval(
    retrieval=rag.Retrieval(
        source=rag.VertexRagStore(
            rag_corpora=[rag_corpus.name],
            similarity_top_k=10,
            vector_distance_threshold=0.5,
        ),
    )
)

### Generate Content with Gemini using Rag Retrieval Tool

In [8]:
# Load tool into Gemini model
rag_gemini_model = GenerativeModel(
    MODEL,
    tools=[rag_retrieval_tool],
)

In [ ]:
response = rag_gemini_model.generate_content("다우 오피스 대쉬보드에 대해서 설명해주세요.")
display(Markdown(response.text))